In [ ]:
import tkinter as tk
import time
import heapq


# Global variables
maze_width = 30
maze_height = 30
square_size = 20
start_row, start_col = None, None
end_row, end_col = None, None
barrier_count = 0
barrier_mode = False
path_taken = []


def set_start():
    global start_row, start_col
    disable_buttons()
    enable_canvas_click(handle_start_click)


def handle_start_click(event):
    global start_row, start_col
    start_row, start_col = event.y // square_size, event.x // square_size
    update_canvas(start_row, start_col, 'light blue')
    disable_canvas_click()
    enable_buttons()


def set_end():
    global end_row, end_col
    disable_buttons()
    enable_canvas_click(handle_end_click)


def handle_end_click(event):
    global end_row, end_col
    end_row, end_col = event.y // square_size, event.x // square_size
    update_canvas(end_row, end_col, 'white')
    disable_canvas_click()
    enable_buttons()


def set_barriers():
    global barrier_mode
    barrier_mode = True
    disable_buttons()
    enable_canvas_click(handle_barrier_click)


def handle_barrier_click(event):
    global barrier_count
    row, col = event.y // square_size, event.x // square_size
    if (row != start_row or col != start_col) and (row != end_row or col != end_col):
        if maze[row][col] != 3:
            maze[row][col] = 3
            barrier_count += 1
            update_canvas(row, col, 'red')


def confirm_barriers():
    global barrier_mode
    barrier_mode = False
    disable_buttons()
    disable_canvas_click()
    enable_buttons()


def disable_buttons():
    btn_set_start.config(state='disabled')
    btn_set_end.config(state='disabled')
    btn_set_barriers.config(state='disabled')
    btn_confirm_barriers.config(state='normal')


def enable_buttons():
    btn_set_start.config(state='normal')
    btn_set_end.config(state='normal')
    btn_set_barriers.config(state='normal')
    btn_confirm_barriers.config(state='disabled')


def enable_canvas_click(callback):
    canvas.bind('<Button-1>', callback)


def disable_canvas_click():
    canvas.unbind('<Button-1>')


def refresh_maze():
    global maze, barrier_count
    maze = [[0] * maze_width for _ in range(maze_height)]
    barrier_count = 0
    for row in range(maze_height):
        for col in range(maze_width):
            update_canvas(row, col, 'black')
    enable_buttons()


def update_canvas(row, col, color):
    canvas.create_rectangle(col * square_size, row * square_size,
                            (col + 1) * square_size, (row + 1) * square_size,
                            fill=color, outline='white')


def update_path_taken(row, col):
    path_text.see(tk.END)
    info = "({},{})".format(row,col)
    path_text.insert(tk.END, info)
    path_text.see(tk.END)
    if (row,col) != (end_row, end_col):
        path_text.insert(tk.END, "->")
def solve_maze(maze):
    path_text.delete(1.0, tk.END)
    cost_text.delete(1.0, tk.END)
    global start_row, start_col, end_row, end_col


    # Run Dijkstra's algorithm to find the shortest path from start to end
    distances = [[float('inf')] * maze_width for _ in range(maze_height)]
    distances[start_row][start_col] = 0
    pq = [(0, start_row, start_col)]
    heapq.heapify(pq)


    # Create a copy of the maze to store the animation state
    maze_copy = [row[:] for row in maze]


    def animate_search():
        if not pq:
            # All cells searched, start pathfinding animation
            animate_pathfinding()
            return


        dist, row, col = heapq.heappop(pq)


        # If the current cell is the end cell, stop the algorithm
        if row == end_row and col == end_col:
            animate_pathfinding()
            return


        # Check neighbors
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            new_row, new_col = row + dx, col + dy
            if 0 <= new_row < maze_height and 0 <= new_col < maze_width and \
                    maze_copy[new_row][new_col] != 3 and \
                    dist + 1 < distances[new_row][new_col]:
                distances[new_row][new_col] = dist + 1
                heapq.heappush(pq, (dist + 1, new_row, new_col))
                # Set the searched boxes to green with a delay
                update_canvas(new_row, new_col, 'green')
                time.sleep(0.1)


        # Schedule the next search animation after a delay
        window.after(1, animate_search)


    def animate_pathfinding():
        # Highlight the shortest path with green color
        path_lst = []
        row, col = end_row, end_col
        count = 0
        while row != start_row or col != start_col:
            path_lst.append((row, col))
            update_canvas(row, col, 'blue')
            count+=1


            min_dist = float('inf')
            next_row, next_col = row, col


            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                new_row, new_col = row + dx, col + dy
                if 0 <= new_row < maze_height and 0 <= new_col < maze_width and \
                        maze[new_row][new_col] != 3 and \
                        distances[new_row][new_col] < min_dist:
                    min_dist = distances[new_row][new_col]
                    next_row, next_col = new_row, new_col


            row, col = next_row, next_col


            # Delay for pathfinding animation
            time.sleep(0.01)


        # Mark the start and end cells
        update_canvas(start_row, start_col, 'yellow')
        update_canvas(end_row, end_col, 'white')
        for i in reversed(path_lst):
            update_path_taken(i[0], i[1])
        cost_text.insert(tk.END, "Cost = {}".format(count))


    def update_canvas(row, col, color):
        x1 = col * square_size
        y1 = row * square_size
        x2 = x1 + square_size
        y2 = y1 + square_size
        canvas.create_rectangle(x1, y1, x2, y2, fill=color, outline='white')
        window.update()


    # Start the search animation
    animate_search()


# Create the main window
window = tk.Tk()
window.title("Maze GUI")
screen_width = window.winfo_screenwidth()
screen_height = window.winfo_screenheight()


# Create the canvas
canvas = tk.Canvas(window, width=maze_width * square_size, height=maze_height * square_size, bg='black')
canvas.pack(side= 'right')


# Create the buttons
btn_set_start = tk.Button(window, text='Set Start', command=set_start)
btn_set_start.pack(side='left')


btn_set_end = tk.Button(window, text='Set End', command=set_end)
btn_set_end.pack(side='left')


btn_set_barriers = tk.Button(window, text='Set Barriers', command=set_barriers)
btn_set_barriers.pack(side='left')


btn_confirm_barriers = tk.Button(window, text='Lock maze', command=confirm_barriers, state='disabled')
btn_confirm_barriers.pack(side='left')


btn_Refresh_maze = tk.Button(window, text='Refresh Maze', command=refresh_maze)
btn_Refresh_maze.pack(side='left')


solve_button = tk.Button(window, text="Solve", command=lambda: solve_maze(maze))
solve_button.pack(side="left")


# Create the textbox
path_text = tk.Text(window, width=160, height=10)
path_text.pack(side= 'top')


cost_text = tk.Text(window, width=80, height=2)
cost_text.pack(side = 'bottom')


# Generate the initial maze
refresh_maze()


# Start the GUI event loop
window.mainloop()